In [1]:
import requests


In [ ]:
response = requests.get('https://static.openfoodfacts.org/data/taxonomies/additives.json', stream=True)

if response.status_code == 200:
    with open('../additives.json', 'wb') as file :
        for chunk in response.iter_content(chunk_size=1000):
            file.write(chunk)
else:
    print(f"Une erreur s'est produite lors du télechargement du fichier")